In [ ]:
import numpy as np
import re

with open('sample.txt', 'r') as f:
  lines = f.read().splitlines()

In [ ]:
def parse_line(line):
  (src, dests) = re.search(r'(.+):\s?(.*)', line).groups()
  return (src, re.split(r'\s+', dests))

rels = list(map(parse_line, lines))

def get_edge_names(rels):
  edges = set()
  for (src, dests) in rels:
    edges.add(src)
    for dest in dests:
      edges.add(dest)
  return sorted(edges)

# Build affinity matrix
def get_affinity_matrix(rels):
  edge_names = get_edge_names(rels)
  A = np.zeros((len(edge_names), len(edge_names)), dtype=int)

  for (src, dests) in rels:
    i_src = edge_names.index(src)
    for dest in dests:
      i_dest = edge_names.index(dest)
      A[i_src, i_dest] = 1
      A[i_dest, i_src] = 1

  return A

In [ ]:
# We'll run Karger's algorithm on the graph to find a cut. Since we know the min-cut is of
# size 3, we can simply repeat the procedure until we find a cut with that exact size.

def contract(G, e):
  i, j = e

  # Merge row/col j into i-th row/col
  G[i, :] += G[j, :]
  G[:, i] += G[:, j]
  G[i, i] = 0

  # Drop j-th row/col from G
  G = np.delete(G, j, axis=0)
  G = np.delete(G, j, axis=1)

  return G

def random_edge(G):
  edges = np.transpose(np.nonzero(G))
  weights = G[edges[:, 0], edges[:, 1]]
  idx = np.random.choice(edges.shape[0], p=(weights / np.sum(weights)))
  return edges[idx]

G_original = get_affinity_matrix(rels)

while True:
  G = G_original.copy()
  vertex_group_counts = np.ones(G.shape[0], dtype=int)

  while G.shape[0] > 2:
    e = random_edge(G)
    G = contract(G, e)

    # Keep track of how many vertices have been merged at each row/col of the
    # graph's affinity matrix; we need this to compute group sizes
    i, j = e
    vertex_group_counts[i] += vertex_group_counts[j]
    vertex_group_counts = np.delete(vertex_group_counts, j)

  min_cut = G[0, 1]
  print(min_cut, vertex_group_counts, np.prod(vertex_group_counts))

  if min_cut == 3:
    break